In [1]:
import pyspark

In [2]:
# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc=SparkContext()
spark = SparkSession(sparkContext=sc)

## Aggregate functions

Two aggregate functions:
   
    

  -aggregate()
   

  -aggregateByKey()

### aggregate(zeroValue,seqOp,combOp)

In [5]:
mt_data=spark.read.csv('mtcars.csv',header=True,inferSchema=True).select(['mpg','disp'])
mt_data.take(4)

[Row(mpg=21.0, disp=160.0),
 Row(mpg=21.0, disp=160.0),
 Row(mpg=22.8, disp=108.0),
 Row(mpg=21.4, disp=258.0)]

In [9]:
mpg_mean=mt_data.select('mpg').rdd.map(lambda x:x[0]).mean()
disp_mean=mt_data.select('disp').rdd.map(lambda x:x[0]).mean()
print('mpg_mean= '+str(mpg_mean)+'\ndisp_mean= '+str(disp_mean))

mpg_mean= 20.090625000000003
disp_mean= 230.721875


In [16]:
#tuple for two columns
zeroValue=(0,0)

In [17]:
#z refers to zeroValue(updated for every run) and x refers to element in Rdd partition
seqOp=lambda z,x:(z[0]+(x[0]-mpg_mean)**2,z[1]+(x[1]-disp_mean)**2)

In [19]:
#aggregates all zero values to one
combOp=lambda px,py:(px[0]+py[0],px[1]+py[1])

In [20]:
mt_data.rdd.aggregate(zeroValue,seqOp,combOp)

(1126.0471874999998, 476184.7946875)

### aggregateByKey(zeroVal,seqOp,combOp)

aggregateByKey merge all results by key

In [23]:
iris_rdd=sc.textFile('Iris.csv')
iris_rdd.take(2)

['Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species',
 '1,5.1,3.5,1.4,0.2,Iris-setosa']

In [24]:
iris_rdd2=iris_rdd.map(lambda x:x.split(',')).filter(lambda x:x[0]!='Id').\
map(lambda x:(x[-1],[*map(float,x[:-1])]))
iris_rdd2.take(2)

[('Iris-setosa', [1.0, 5.1, 3.5, 1.4, 0.2]),
 ('Iris-setosa', [2.0, 4.9, 3.0, 1.4, 0.2])]

In [25]:
#defining the function
zeroValue=(0,0)
seqOp=(lambda x,y:(x[0]+(y[0])**2 ,x[1]+(y[1])**2))
combOp=(lambda x,y:(x[0]+y[0],x[1]+y[1]))

In [27]:
iris_rdd2.aggregateByKey(zeroValue,seqOp,combOp).collect()

[('Iris-setosa', (42925.0, 1259.0899999999997)),
 ('Iris-virginica', (797925.0, 2189.9000000000005)),
 ('Iris-versicolor', (295425.0, 1774.8600000000001))]